# improve GA

In [1]:
from time import *
from Initia_Oper import Initialization
import numpy as np
import pandas as pd
from Fitness_pro_sele import FPS
from Crossover_Uniform import uniform_crossover_operator
from Mutation import Standard_Bit_Mutation
#from Improve_pro import Polynomial_Algorithm
from sklearn.svm import SVR 
def improv_GGA(X_train,Y_train,X_test,Y_test,T,N,s,P_cross,finsh_label,s_max,c,Gamma,Epsilon):
    t=1
    Population=[]
    Sample_train_num,feature_num=X_train.shape
    Sample_test_num,feature_num=X_test.shape
    clf=SVR(kernel='rbf',C=c,gamma=Gamma,epsilon=Epsilon)
    best_active=[]
    best_SIC=float("inf")
    best_MSE=float("inf")
    finish_count_num=0
    Min_Fitness=[]
    Max_Fitness=[]
    Mean_Fitness=[]
    #初始化
    for indi in range(N):
        Individual=Initialization(s,feature_num)
        impro_individual=Polynomial_Algorithm(Individual,X_train,Y_train,s_max)
        Population.append(impro_individual)
    total_ind=N
    old_fit=float("inf")
    while t<=T:
        #calculate Fitness
        Fitness=[]
        for each_fit in range(total_ind):
            Active_set=[]
            for fea in range(feature_num):
                if Population[each_fit][fea]==1:
                    Active_set.append(fea)
            clf.fit(X_train[:,Active_set],Y_train)
            Y_predict=clf.predict(X_test[:,Active_set]).reshape(Sample_test_num,1)
            MSE=np.sum((Y_predict-Y_test)**2)/Sample_test_num
            if MSE<best_MSE:
                best_active=Active_set
                best_MSE=MSE
            Fitness.append(MSE)
            print(MSE)
        Min_Fitness.append(np.min(Fitness))
        Max_Fitness.append(np.max(Fitness))
        Mean_Fitness.append(np.mean(Fitness))
        if np.min(Fitness)<old_fit:
            old_fit=np.min(Fitness)
            finish_count_num=0
        else:
            finish_count_num+=1
            old_fit=np.min(Fitness)
        if finish_count_num>=finsh_label:
            break
        new_population=[]
        new_indi_num=0
        opt_indi=Population[np.argmin(Fitness)]
        new_population.append(opt_indi)
        new_indi_num+=1
        while new_indi_num<N:
        # Selection Operator
            Parent1=Population[FPS(Fitness,total_ind)]
            Parent2=Population[FPS(Fitness,total_ind)]
              
        #crossover Operator
            P_0=[P_cross for i in range(feature_num)]
            offspring1=uniform_crossover_operator(Parent1,Parent2,P_0)
            offspring2=uniform_crossover_operator(Parent1,Parent2,P_0)
            impro_off1=Polynomial_Algorithm(offspring1,X_train,Y_train,s_max)
            impro_off2=Polynomial_Algorithm(offspring2,X_train,Y_train,s_max)
            new_population.append(impro_off1)
            new_population.append(impro_off2) 
            new_indi_num+=2
        Population=new_population
        t+=1
        total_ind=len(Population)
    return best_active,best_MSE,Min_Fitness,Max_Fitness,Mean_Fitness

# splicing method

In [2]:
import numpy as np
import copy
def Polynomial_Algorithm(Individual,X_train,Y_train,s_max):
    new_Active_set=[]
    new_Inactive_set=[]
    sample_num,feature_num=X_train.shape
    for i in range(feature_num):
        if Individual[i]==1:
            new_Active_set.append(i)
        else:
            new_Inactive_set.append(i)
    if len(new_Active_set)<=s_max:
        Active_set=new_Active_set
        Inactive_set=new_Inactive_set
        tau=0.01*len(Active_set)*np.log(feature_num)*np.log(np.log(sample_num))/sample_num
        beta=np.dot(np.dot(np.linalg.pinv(np.dot(X_train[:,Active_set].T,X_train[:,Active_set])),X_train[:,Active_set].T),Y_train)
        erro=Y_train-np.dot(X_train[:,Active_set],beta)
        d=np.dot(X_train[:,Inactive_set].T,erro)/sample_num
    else:
        new_beta=np.dot(np.dot(np.linalg.pinv(np.dot(X_train[:,new_Active_set].T,X_train[:,new_Active_set])),X_train[:,new_Active_set].T),Y_train)
        backward_sacrifice_lis=[]
        for i in range(len(new_Active_set)):
            backward_sacrifice=np.dot(X_train[:,new_Active_set[i]],X_train[:,new_Active_set[i]])/(2*sample_num)*(new_beta[i,0])**2
            backward_sacrifice_lis.append((backward_sacrifice,new_Active_set[i]))
        backward_sacrifice_lis.sort(key=lambda backward_sacrifice_lis:backward_sacrifice_lis[0],reverse=True)
        Active_set=[]
        for twin_ele in backward_sacrifice_lis[:s_max]:
            Active_set.append(twin_ele[1])
        Inactive_set=[i for i in range(feature_num)]
        for de_i in Active_set:
            Inactive_set.remove(de_i)
        tau=0.01*len(Active_set)*np.log(feature_num)*np.log(np.log(sample_num))/sample_num
        beta=np.dot(np.dot(np.linalg.pinv(np.dot(X_train[:,Active_set].T,X_train[:,Active_set])),X_train[:,Active_set].T),Y_train)
        erro=Y_train-np.dot(X_train[:,Active_set],beta)
        d=np.dot(X_train[:,Inactive_set].T,erro)/sample_num
    while True:
        L=L_0=np.sum((Y_train-np.dot(X_train[:,Active_set],beta))**2)/(2*sample_num)
        backward_sacrifice_lis=[]
        forward_sacrifice_lis=[]
        active_len=len(Active_set)
        for i in range(active_len):
            backward_sacrifice=np.dot(X_train[:,Active_set[i]],X_train[:,Active_set[i]])/(2*sample_num)*(beta[i,0])**2
            backward_sacrifice_lis.append((backward_sacrifice,Active_set[i]))
        for j in range(len(Inactive_set)):
            mid=np.dot(X_train[:,Inactive_set[j]],X_train[:,Inactive_set[j]])
            forward_sacrifice=(mid/(2*sample_num))*(d[j,0]/(mid/sample_num))**2
            forward_sacrifice_lis.append((forward_sacrifice,Inactive_set[j]))
        #从小到大排位    
        backward_sacrifice_lis.sort(key=lambda backward_sacrifice_lis:backward_sacrifice_lis[0])
        #从大到小排位
        forward_sacrifice_lis.sort(key=lambda forward_sacrifice_lis:forward_sacrifice_lis[0],reverse=True)
        for i in range(len(backward_sacrifice_lis)):
            Active_set[i]=backward_sacrifice_lis[i][1]
        for i in range(len(forward_sacrifice_lis)):
            Inactive_set[i]=forward_sacrifice_lis[i][1]
        #记录在k取值多少,最佳数据
        trace_activ_set,trace_inactiv_set,trace_beta,trace_d=[],[],0,0
        #k_max设置范围
        for k in range(1,min(len(Active_set)+1,len(Inactive_set)+1)):
            new_active_set,new_inactive_set=copy.deepcopy(Active_set),copy.deepcopy(Inactive_set)
            new_active_set[:k],new_inactive_set[:k]=new_inactive_set[:k],new_active_set[:k]
            new_active_set,new_inactive_set=sorted(new_active_set),sorted(new_inactive_set)
            new_beta=np.dot(np.dot(np.linalg.pinv(np.dot(X_train[:,new_active_set].T,X_train[:,new_active_set])),X_train[:,new_active_set].T),Y_train) 
            new_erro=Y_train-np.dot(X_train[:,new_active_set],new_beta)
            new_d=np.dot(X_train[:,new_inactive_set].T,erro)/sample_num
            L_new=np.sum((Y_train-np.dot(X_train[:,new_active_set],new_beta))**2)/(2*sample_num)
            if L_new<L:
                #print(L_new)
                trace_activ_set,trace_inactiv_set,trace_beta,trace_d=new_active_set,new_inactive_set,new_beta,new_d
                L=L_new
                #print(L,len(new_active_set))
        #print(L_0,L)
        if trace_activ_set:
            beta,d,Active_set,Inactive_set=trace_beta,trace_d,trace_activ_set,trace_inactiv_set 
        if L_0-L<=tau:
            break
        

    merge_indi=[0 for i in range(feature_num)]
    for index_ in Active_set:
        merge_indi[index_]=1
    return merge_indi

In [12]:
import pandas as pd
import numpy as np
data=pd.read_csv("HuSheepData.csv",encoding='gb18030')
Weaning_Weight=np.array(data)[5,1:].astype(np.float64).reshape(240,1)
Birth_Weight=np.array(data)[4,1:].astype(np.float64).reshape(240,1)
Six_Weight=np.array(data)[6,1:].astype(np.float64).reshape(240,1)
Featur_Sample=np.array(data)[8:,1:].astype(np.float64).T
X_train=Featur_Sample[:170,:]
X_test=Featur_Sample[170:,:]
Birth_Weight_train=Birth_Weight[:170]
Birth_Weight_test=Birth_Weight[170:]

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (1,2,3,4,5,7,9,10,11,13,14,15,16,17,18,20,21,22,23,24,27,28,31,34,74,95,96,97,98,99,100,101,102,103,104,105,106,107,108,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,139,141,143,146,148,150,152,153,155,167,169,170,171,172,173,174,175,176,177,178,179,180,191,192,193,194,205,206,207,210,216,217,218,219,220,221,222,223,224,225,226,227,228) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
Featur_Sample.shape

(240, 54183)

In [5]:
S_max_lis=[i for i in range(10,60,10)]
P_uniform_lis=[0.1,0.2,0.3,0.4,0.5]


In [6]:
best_active_lis=[]
best_MSE_lis=[]
min_fitness_lis=[]
max_fitness_lis=[]
mean_fitness_lis=[]
trace_S_max_lis=[]
trace_p_unifor_lis=[]
for s_max in S_max_lis:
    for p_uniform in P_uniform_lis:
        best_active,best_MSE,Min_Fitness,Max_Fitness,Mean_Fitness=improv_GGA(X_train,Birth_Weight_train,X_test,Birth_Weight_test,30,20,2000,p_uniform,5,s_max,0.1,0.01,0.00001)
        best_active_lis.append(best_active)
        best_MSE_lis.append(best_MSE)
        min_fitness_lis.append(Min_Fitness)
        max_fitness_lis.append(Max_Fitness)
        mean_fitness_lis.append(Mean_Fitness)
        trace_S_max_lis.append(s_max)
        trace_p_unifor_lis.append(p_uniform)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: 

0.23026089636534205
0.23002347844519155
0.23142002706670514
0.23042260094054215
0.23080382609108888
0.22924282876435909

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 


0.22945485867860807
0.22959870909129962
0.22905560991214183
0.23245447282392442
0.22963018294365645
0.22866666330125274


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

...

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: 

0.2260501061316457
0.22905228644610873
0.23039066495844146
0.23017062572315794
0.230273329751642
0.2260501061316457
0.2260501061316457
0.22991260593877588
0.22794247858852063
0.22938809636408253


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

0.2268204823905475
0.2277534110200157
0.23026525344597154
0.23422163550953842
0.23391038820703675
0.23265011758001128
0.23394903269017686
0.2260501061316457
0.2260501061316457
0.2300867004127822
0.23059815214223575


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [15]:
np.argmin(best_MSE_lis),best_MSE_lis[14],len(best_active_lis[14]),trace_S_max_lis[14],trace_p_unifor_lis[14]

(14, 0.221294305859685, 28, 30, 0.5)

[0.22787854980576192,
 0.22662627625858883,
 0.22616432349989263,
 0.22616432349989263,
 0.2253632251426552,
 0.22406063029227188,
 0.22406063029227188,
 0.22217715566831145,
 0.22217715566831145,
 0.22170664021595343,
 0.22166363289092036,
 0.22166363289092036,
 0.22134091371911438,
 0.22134091371911438,
 0.221294305859685,
 0.221294305859685,
 0.221294305859685,
 0.221294305859685,
 0.221294305859685,
 0.221294305859685]

In [19]:
fitness_lis=[]
fitness_lis.append(mean_fitness_lis[14])
fitness_frame=pd.DataFrame(fitness_lis)
fitness_frame.to_csv("Birth_weight_propos.csv")

In [21]:
fitness_lis=[]
fitness_lis.append(min_fitness_lis[14])
fitness_frame=pd.DataFrame(fitness_lis)
fitness_frame.to_csv("Birth_weight_propos.csv")

In [13]:
data_lis=[]
data_lis.append(trace_S_max_lis)
data_lis.append(trace_p_unifor_lis)
data_lis.append(best_MSE_lis)
import pandas as pd
data_birth_csv=pd.DataFrame(data_lis)
data_birth_csv.to_csv("Birth_weight_para.csv")

In [39]:
col=np.array(data)[:,0]
gene_name=col[8:]

In [47]:

gene=pd.DataFrame(gene_name[best_active_lis[14]])
gene.to_csv("best_gene_birth_pro.csv")